# Table 3. Factors associated with persisting untreated pain (Secondary transport population, all ages)

In [ ]:
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
import warnings
from sklearn.preprocessing import StandardScaler
from utils.utils import _extract_venous_access_features
warnings.filterwarnings('ignore')

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/rega_data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/rega_data/rega_physician_list_09102025.xlsx'
meta_medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/medreg_extraction/joined_final_complete_extractions_20251008_221735.xlsx'
restrict_to_secondary = True

In [ ]:
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)
meta_medic_df = pd.read_excel(meta_medic_data_path)

medic_df['full_name'] = medic_df['Mitglieder mit Einsatzfunktion'].str.replace(' (Flugarzt/Flugärztin)', '')
medic_df.drop_duplicates(subset=['Mitglieder mit Einsatzfunktion'], inplace=True)
medic_df = medic_df.merge(meta_medic_df, how='left', on='full_name')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

data_df = data_df.drop_duplicates(subset=['SNZ Ereignis Nr. '])
data_df = data_df[data_df['VAS_on_scene'] > 3]

n_missing_arrival = data_df['VAS_on_arrival'].isna().sum()
print(f'Excluded {n_missing_arrival} patients with missing VAS_on_arrival')
data_df = data_df.dropna(subset=['VAS_on_arrival'])

if restrict_to_secondary:
    n_primary = data_df[data_df['Einsatzart'] != 'Sekundär'].shape[0]
    print(f'Excluded {n_primary} primary transport patients')
    data_df = data_df[data_df['Einsatzart'] == 'Sekundär']

if 'NACA' not in data_df.columns and 'NACA (nummerisch)' in data_df.columns:
    data_df['NACA'] = data_df['NACA (nummerisch)']

In [ ]:
def univariate_logistic_regression(df, outcome_var, predictor_vars):
    results = []
    for var in predictor_vars:
        X = df[[var]].copy()
        y = df[outcome_var]
        X_with_const = sm.add_constant(X)
        try:
            model = sm.Logit(y, X_with_const).fit(disp=0)
            coef = model.params[var]
            or_value = np.exp(coef)
            ci_lower = np.exp(model.conf_int().loc[var, 0])
            ci_upper = np.exp(model.conf_int().loc[var, 1])
            p_value = model.pvalues[var]
            results.append({
                'Variable': var,
                'Coefficient': coef,
                'OR': or_value,
                'CI_lower': ci_lower,
                'CI_upper': ci_upper,
                'P_value': p_value,
                'OR_CI': f'{or_value:.2f} ({ci_lower:.2f}-{ci_upper:.2f})',
                'P_formatted': f'{p_value:.3f}' if p_value >= 0.001 else '<0.001',
            })
        except Exception as e:
            print(f'Error with variable {var}: {e}')
    return pd.DataFrame(results)

def multivariate_logistic_regression(df, outcome_var, predictor_vars, normalize=False):
    X = df[predictor_vars].copy()
    y = df[outcome_var]
    if normalize:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    X_with_const = sm.add_constant(X)
    model = sm.Logit(y, X_with_const).fit(disp=0)
    results = []
    for var in X.columns:
        coef = model.params[var]
        or_value = np.exp(coef)
        ci_lower = np.exp(model.conf_int().loc[var, 0])
        ci_upper = np.exp(model.conf_int().loc[var, 1])
        p_value = model.pvalues[var]
        results.append({
            'Variable': var,
            'Coefficient': coef,
            'OR': or_value,
            'CI_lower': ci_lower,
            'CI_upper': ci_upper,
            'P_value': p_value,
            'OR_CI': f'{or_value:.2f} ({ci_lower:.2f}-{ci_upper:.2f})',
            'P_formatted': f'{p_value:.3f}' if p_value >= 0.001 else '<0.001',
        })
    return pd.DataFrame(results), model

def drop_zero_variance(df, vars_list):
    kept = [v for v in vars_list if df[v].nunique(dropna=True) > 1]
    dropped = [v for v in vars_list if v not in kept]
    if dropped:
        print(f'Dropped zero-variance variables: {dropped}')
    return kept

In [ ]:
df = data_df.copy()
df['insufficient_pain_mgmt'] = (df['VAS_on_arrival'] > 3).astype(int)

df['age'] = df['Alter ']
df['male_patient'] = (df['Geschlecht'] == 'Männlich').astype(int)
df['male_physician'] = (df['physician_sex'] == 'm').astype(int)

df['event_year'] = pd.to_datetime(df['Ereignisdatum'], format='%d.%m.%Y').dt.year
df['physician_age'] = df['event_year'] - df['year_of_birth']
df['physician_licence_year'] = df['licence_date'].apply(lambda x: str(x).split('.')[-1] if '.' in str(x) else str(x))
df['physician_experience_years'] = df['event_year'] - pd.to_numeric(df['physician_licence_year'], errors='coerce')

df['physician_anesthesiologist'] = df['specialist_qualifications'].str.contains('Anaesthesiology', na=False).astype(int)
df['physician_intensivist'] = df['specialist_qualifications'].str.contains('Intensive care medicine', na=False).astype(int)
df['physician_internist'] = df['specialist_qualifications'].str.contains('General Internal Medicine|General medical practitioner', na=False).astype(int)

df['primary_mission'] = (df['Einsatzart'] == 'Primär').astype(int)
df['night_mission'] = (df['Tag oder Nacht'] == 'Nacht').astype(int)
df['winter_season'] = np.where(df['Monat'].isin(['Oktober', 'November', 'Dezember', 'Januar', 'Februar', 'März']), 1, 0).astype(int)
df['winch_extraction'] = df['Bergungen'].str.contains('Winde', na=False).astype(int)
df['vas_scene'] = df['VAS_on_scene']
df['mission_duration'] = (pd.to_datetime(df['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - pd.to_datetime(df['Erstbefund'], format='%d.%m.%Y %H:%M:%S')).dt.total_seconds() / 60

venous_access_features = _extract_venous_access_features(df['Zugänge'])
df = pd.concat([df, venous_access_features], axis=1)
df['no_venous_access'] = (df['venous_access_count'] == 0).astype(int)

df['fentanyl_dose'] = 0
df['ketamine_dose'] = 0
df['esketamine_dose'] = 0
df['morphine_dose'] = 0
df['Alle Medikamente'] = df['Alle Medikamente'].str.replace(',', ';')
for i, row in df.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        if '7IE' in analgetic:
            continue
        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        if 'Fentanyl' in analgetic and '/h' not in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip()
            df.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Fentanyl' in analgetic and '/h' in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
            dose = float(dose) * df.at[i, 'mission_duration']
            df.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
            dose = analgetic.split('Ketamin')[-1].strip()
            df.at[i, 'ketamine_dose'] += float(dose)
        elif 'Esketamin' in analgetic:
            dose = analgetic.split('Esketamin')[-1].strip()
            df.at[i, 'esketamine_dose'] += float(dose)
        elif 'Morphin' in analgetic or 'Morphine' in analgetic:
            dose = analgetic.split('Morphin')[-1].strip()
            df.at[i, 'morphine_dose'] += float(dose)

df['any_opiate_dose'] = df['morphine_dose'] + df['fentanyl_dose']
df['any_ketamine_dose'] = df['ketamine_dose'] + df['esketamine_dose']
df['any_opiate_given'] = (df['morphine_dose'] > 0) | (df['fentanyl_dose'] > 0)
df['any_ketamine_given'] = (df['ketamine_dose'] > 0) | (df['esketamine_dose'] > 0)
df['no_analgesic'] = ((df['any_opiate_given'] == 0) & (df['any_ketamine_given'] == 0)).astype(int)
df['persisting_untreated_pain'] = ((df['insufficient_pain_mgmt'] == 1) & (df['no_analgesic'] == 1)).astype(int)

model_vars = [
    'persisting_untreated_pain', 'age', 'NACA', 'male_patient', 'male_physician',
    'physician_age', 'physician_experience_years', 'physician_anesthesiologist',
    'physician_intensivist', 'physician_internist', 'mission_duration', 'primary_mission',
    'night_mission', 'winter_season', 'winch_extraction', 'vas_scene',
    'venous_access_count', 'no_venous_access'
]
model_vars = [v for v in model_vars if v in df.columns]
df_clean = df[model_vars].dropna()
predictor_vars = [v for v in model_vars if v != 'persisting_untreated_pain']
predictor_vars = drop_zero_variance(df_clean, predictor_vars)

print(f'Secondary patients included: {len(df_clean)}')
print(f"Outcome rate (persisting untreated pain): {df_clean['persisting_untreated_pain'].mean():.1%}")

In [ ]:
univariate_results = univariate_logistic_regression(df_clean, 'persisting_untreated_pain', predictor_vars)
multivariate_results, multivariate_model = multivariate_logistic_regression(df_clean, 'persisting_untreated_pain', predictor_vars)

univariate_results.sort_values('P_value')

In [ ]:
multivariate_results.sort_values('P_value')

In [ ]:
multivariate_results_normalized, _ = multivariate_logistic_regression(
    df_clean, 'persisting_untreated_pain', predictor_vars, normalize=True
 )

summary_table = pd.DataFrame({
    'Variable': univariate_results['Variable'],
    'Univariate OR (95% CI)': univariate_results['OR_CI'],
    'Univariate P-value': univariate_results['P_formatted'],
})

summary_table = summary_table.merge(
    multivariate_results[['Variable', 'OR_CI', 'P_formatted']],
    on='Variable', how='left'
).rename(columns={
    'OR_CI': 'Multivariate OR (95% CI)',
    'P_formatted': 'Multivariate P-value',
})

summary_table = summary_table.merge(
    multivariate_results_normalized[['Variable', 'OR_CI', 'P_formatted']],
    on='Variable', how='left'
).rename(columns={
    'OR_CI': 'Normalized OR (95% CI)',
    'P_formatted': 'Normalized P-value',
})

summary_table

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Figure: Forest Plot - Secondary transport population
def create_secondary_forest_plot():
    """
    Create a forest plot showing odds ratios for secondary transport population only
    """
    fig, ax = plt.subplots(figsize=(12, 10))

    # Variable labels for better presentation
    var_labels = {
        'age': 'Patient age (years)',
        'male_patient': 'Male patient',
        'male_physician': 'Male physician',
        'physician_age': 'Physician age (years)',
        'physician_experience_years': 'Physician experience (years)',
        'physician_anesthesiologist': 'Physician anesthesiologist',
        'physician_intensivist': 'Physician intensivist',
        'physician_internist': 'Physician internist',
        'mission_duration': 'Mission duration (minutes)',
        'NACA': 'NACA score',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'winter_season': 'Winter season',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'NRS on scene',
        'venous_access_count': 'Number of venous accesses',
        'no_venous_access': 'No venous access',
    }

    if len(multivariate_results) > 0:
        # Prepare data from secondary transport multivariate results
        variables = []
        ors = []
        ci_lower = []
        ci_upper = []
        pvals = []

        # Sort by OR magnitude for better visualization
        secondary_results_sorted = multivariate_results.sort_values('OR', ascending=True)

        for _, row in secondary_results_sorted.iterrows():
            var = row['Variable']
            variables.append(var_labels.get(var, var))
            ors.append(row['OR'])
            ci_lower.append(row['CI_lower'])
            ci_upper.append(row['CI_upper'])
            pvals.append(row['P_value'])

        # Filter out non-finite estimates to avoid invalid axis limits
        variables = np.array(variables, dtype=object)
        ors = np.array(ors, dtype=float)
        ci_lower = np.array(ci_lower, dtype=float)
        ci_upper = np.array(ci_upper, dtype=float)
        pvals = np.array(pvals, dtype=float)
        finite_mask = np.isfinite(ors) & np.isfinite(ci_lower) & np.isfinite(ci_upper)
        if not finite_mask.all():
            dropped = variables[~finite_mask].tolist()
            print(f'Excluded non-finite estimates: {dropped}')
        variables = variables[finite_mask].tolist()
        ors = ors[finite_mask].tolist()
        ci_lower = ci_lower[finite_mask].tolist()
        ci_upper = ci_upper[finite_mask].tolist()
        pvals = pvals[finite_mask].tolist()

        if len(variables) == 0:
            print('No finite estimates available for forest plot')
            return fig

        y_positions = np.arange(len(variables))

        # Plot the forest plot
        for i, (or_val, ci_l, ci_u, p_val) in enumerate(zip(ors, ci_lower, ci_upper, pvals)):
            color = '#FF4444' if p_val < 0.05 else '#666666'
            size = 120 if p_val < 0.05 else 80

            # Plot point estimate
            ax.scatter(or_val, y_positions[i], color=color, s=size, zorder=3, alpha=0.8)

            # Plot confidence interval
            ax.plot([ci_l, ci_u], [y_positions[i], y_positions[i]],
                   color=color, linewidth=2, alpha=0.7, zorder=2)

            # Plot CI caps
            cap_size = 0.1
            ax.plot([ci_l, ci_l], [y_positions[i]-cap_size, y_positions[i]+cap_size],
                   color=color, linewidth=2, alpha=0.7, zorder=2)
            ax.plot([ci_u, ci_u], [y_positions[i]-cap_size, y_positions[i]+cap_size],
                   color=color, linewidth=2, alpha=0.7, zorder=2)

            # Add OR and CI text
            ax.text(max(ors) * 1.3, y_positions[i],
                   f'{or_val:.2f} ({ci_l:.2f}-{ci_u:.2f})',
                   va='center', fontsize=9, fontweight='bold' if p_val < 0.05 else 'normal')

        # Reference line at OR = 1
        ax.axvline(x=1, color='black', linestyle='--', linewidth=1, alpha=0.8)

        # Formatting
        ax.set_xlabel('Odds Ratio (95% CI)', fontsize=14, fontweight='bold')
        ax.set_yticks(y_positions)
        ax.set_yticklabels(variables, fontsize=11)

        # Set x-axis limits with some padding
        x_min = min(min(ci_lower) * 0.8, 0.1)
        x_max = max(max(ci_upper), max(ors)) * 1.3
        ax.set_xlim(x_min, x_max)

        # Add grid
        ax.grid(True, alpha=0.3, axis='x')

        # Add model info text
        model_info = f"""
        Model: n = {len(df_clean)} secondary transport patients
        Outcome: Persisting untreated pain (VAS arrival > 3 and no analgesic)
        """
        # ax.text(1.02, 0.98, model_info.strip(), transform=ax.transAxes,
        #        fontsize=9, verticalalignment='top',
        #        bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))

        plt.tight_layout()
        plt.show()

        # Print summary
        print('\nForest Plot Summary - Secondary transport population:')
        print('=' * 50)
        print(f'Total variables analyzed: {len(variables)}')
        print(f'Significant associations (p < 0.05): {sum(1 for p in pvals if p < 0.05)}')
        print(f'Sample size: {len(df_clean)} secondary transport patients')
        print(f"Outcome rate: {df_clean['persisting_untreated_pain'].mean():.1%}")

        # List significant variables
        sig_vars = [(var, or_val, p_val) for var, or_val, p_val in zip(variables, ors, pvals) if p_val < 0.05]
        if sig_vars:
            print('\nSignificant associations:')
            for var, or_val, p_val in sig_vars:
                direction = 'increased' if or_val > 1 else 'decreased'
                print(f'  • {var}: OR = {or_val:.2f} ({direction} odds, p = {p_val:.3f})')
        else:
            print('\nNo variables reached statistical significance (p < 0.05)')
    else:
        print('No secondary transport multivariate results available for forest plot')

    return fig

# Create the secondary transport forest plot
fig = create_secondary_forest_plot()

In [ ]:
# Forest plot using normalized data for secondary transport population only

def create_secondary_forest_plot_normalized():
    """
    Create a forest plot showing odds ratios for secondary transport population only
    using normalized predictors (per 1 SD increase).
    """
    fig, ax = plt.subplots(figsize=(12, 10))

    # Variable labels for better presentation
    var_labels = {
        'age': 'Patient age (years)',
        'male_patient': 'Male patient',
        'male_physician': 'Male physician',
        'physician_age': 'Physician age (years)',
        'physician_experience_years': 'Physician experience (years)',
        'physician_anesthesiologist': 'Physician anesthesiologist',
        'physician_intensivist': 'Physician intensivist',
        'physician_internist': 'Physician internist',
        'mission_duration': 'Mission duration (minutes)',
        'NACA': 'NACA score',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'winter_season': 'Winter season',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'NRS on scene',
        'venous_access_count': 'Number of venous accesses',
        'no_venous_access': 'No venous access',
    }

    if len(multivariate_results_normalized) > 0:
        # Prepare data from normalized secondary transport multivariate results
        variables = []
        ors = []
        ci_lower = []
        ci_upper = []
        pvals = []

        # Sort by OR magnitude for better visualization
        secondary_results_sorted = multivariate_results_normalized.sort_values('OR', ascending=True)

        for _, row in secondary_results_sorted.iterrows():
            var = row['Variable']
            variables.append(var_labels.get(var, var))
            ors.append(row['OR'])
            ci_lower.append(row['CI_lower'])
            ci_upper.append(row['CI_upper'])
            pvals.append(row['P_value'])

        # Filter out non-finite estimates to avoid invalid axis limits
        variables = np.array(variables, dtype=object)
        ors = np.array(ors, dtype=float)
        ci_lower = np.array(ci_lower, dtype=float)
        ci_upper = np.array(ci_upper, dtype=float)
        pvals = np.array(pvals, dtype=float)
        finite_mask = np.isfinite(ors) & np.isfinite(ci_lower) & np.isfinite(ci_upper)
        if not finite_mask.all():
            dropped = variables[~finite_mask].tolist()
            print(f'Excluded non-finite estimates: {dropped}')
        variables = variables[finite_mask].tolist()
        ors = ors[finite_mask].tolist()
        ci_lower = ci_lower[finite_mask].tolist()
        ci_upper = ci_upper[finite_mask].tolist()
        pvals = pvals[finite_mask].tolist()

        if len(variables) == 0:
            print('No finite estimates available for normalized forest plot')
            return fig

        y_positions = np.arange(len(variables))

        # Plot the forest plot
        for i, (or_val, ci_l, ci_u, p_val) in enumerate(zip(ors, ci_lower, ci_upper, pvals)):
            color = '#FF4444' if p_val < 0.05 else '#666666'
            size = 120 if p_val < 0.05 else 80

            # Plot point estimate
            ax.scatter(or_val, y_positions[i], color=color, s=size, zorder=3, alpha=0.8)

            # Plot confidence interval
            ax.plot([ci_l, ci_u], [y_positions[i], y_positions[i]],
                   color=color, linewidth=2, alpha=0.7, zorder=2)

            # Plot CI caps
            cap_size = 0.1
            ax.plot([ci_l, ci_l], [y_positions[i]-cap_size, y_positions[i]+cap_size],
                   color=color, linewidth=2, alpha=0.7, zorder=2)
            ax.plot([ci_u, ci_u], [y_positions[i]-cap_size, y_positions[i]+cap_size],
                   color=color, linewidth=2, alpha=0.7, zorder=2)

            # Add OR and CI text
            ax.text(max(ors) * 1.2, y_positions[i],
                   f'{or_val:.2f} ({ci_l:.2f}-{ci_u:.2f})',
                   va='center', fontsize=9, fontweight='bold' if p_val < 0.05 else 'normal')

        # Reference line at OR = 1
        ax.axvline(x=1, color='black', linestyle='--', linewidth=1, alpha=0.8)

        # Formatting
        ax.set_xlabel('Odds Ratio per 1 SD increase (95% CI)', fontsize=14, fontweight='bold')
        ax.set_yticks(y_positions)
        ax.set_yticklabels(variables, fontsize=11)

        # Set x-axis limits with some padding
        x_min = min(min(ci_lower) * 0.8, 0.1)
        x_max = max(max(ci_upper), max(ors)) * 1.3
        ax.set_xlim(x_min, x_max)

        # Add grid
        ax.grid(True, alpha=0.3, axis='x')

        # remove upper and right spine
        ax.spines[['right', 'top']].set_visible(False)

        plt.tight_layout()
        plt.show()

        # Print summary
        print('\nForest Plot Summary - Secondary transport population (Normalized):')
        print('=' * 50)
        print(f'Total variables analyzed: {len(variables)}')
        print(f'Significant associations (p < 0.05): {sum(1 for p in pvals if p < 0.05)}')
        print(f'Sample size: {len(df_clean)} secondary transport patients')
        print(f"Outcome rate: {df_clean['persisting_untreated_pain'].mean():.1%}")

        # List significant variables
        sig_vars = [(var, or_val, p_val) for var, or_val, p_val in zip(variables, ors, pvals) if p_val < 0.05]
        if sig_vars:
            print('\nSignificant associations:')
            for var, or_val, p_val in sig_vars:
                direction = 'increased' if or_val > 1 else 'decreased'
                print(f'  • {var}: OR = {or_val:.2f} ({direction} odds, p = {p_val:.3f})')
        else:
            print('\nNo variables reached statistical significance (p < 0.05)')
    else:
        print('No secondary transport normalized multivariate results available for forest plot')

    return fig

# Create the secondary transport normalized forest plot
norm_fig = create_secondary_forest_plot_normalized()

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt

output_dir = Path('/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/secondary_analgesia')
output_dir.mkdir(parents=True, exist_ok=True)
notebook_tag = 'table3_persisting_untreated_pain_secondary'

# Save summary table
if 'summary_table' in globals():
    try:
        summary_table.to_excel(output_dir / f'{notebook_tag}_summary_table.xlsx', index=False)
    except Exception as exc:
        print(f'Could not save Excel summary table: {exc}')

# Save named figures if present
saved_fig_nums = set()
if 'fig' in globals():
    saved_fig_nums.add(fig.number)
    fig.savefig(output_dir / f'{notebook_tag}_forest_plot.png', dpi=300, bbox_inches='tight')
if 'norm_fig' in globals():
    saved_fig_nums.add(norm_fig.number)
    norm_fig.savefig(output_dir / f'{notebook_tag}_forest_plot_normalized.png', dpi=300, bbox_inches='tight')

# Save any other open matplotlib figures
for idx, num in enumerate(plt.get_fignums(), start=1):
    if num in saved_fig_nums:
        continue
    plt.figure(num).savefig(output_dir / f'{notebook_tag}_figure_{idx}.png', dpi=300, bbox_inches='tight')